To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

[NEW] Llama-3.1 8b, 70b & 405b are trained on a crazy 15 trillion tokens with 128K long context lengths!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install datasets

In [3]:
import random
import re
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from datasets import Dataset
import pandas as pd
import numpy as np
#from unsloth import FastLanguageModel
import torch
from sklearn.model_selection import train_test_split
from google.colab import userdata

In [4]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
#load_model = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
load_model = "/content/drive/MyDrive/LLM Projects/Kaggle/EssayGrader/Llama_Annotator_v1/checkpoint-380"


In [5]:
all_data_df = pd.read_csv('/content/drive/MyDrive/LLM Projects/Kaggle/Prepared Data/all_data.csv')
arguments_df = pd.read_csv('/content/drive/MyDrive/LLM Projects/Kaggle/Prepared Data/scores_arguments.csv')

#remove nan
arguments_df = arguments_df.dropna()

#remove duplicates
arguments_df = arguments_df.drop_duplicates()

In [6]:
train_data, eval_data = train_test_split(arguments_df, test_size=0.2, random_state=42, stratify=arguments_df['holistic_essay_score'])

# Print info about the sampled data
print(type(train_data))
print(f"Total training samples: {len(train_data)}")
print(train_data['holistic_essay_score'].value_counts().sort_index())

print(type(eval_data))
print(f"Total evaluation samples: {len(eval_data)}")
print(eval_data['holistic_essay_score'].value_counts().sort_index())

<class 'pandas.core.frame.DataFrame'>
Total training samples: 3245
holistic_essay_score
1.0    430
2.0    988
3.0    843
4.0    387
5.0    412
6.0    185
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Total evaluation samples: 812
holistic_essay_score
1.0    108
2.0    247
3.0    211
4.0     97
5.0    103
6.0     46
Name: count, dtype: int64


## Prompt Templates

In [7]:
def create_train_prompt(example, include_annotations=random.choice([True, False])):

    full_text = example['full_text']
    annotated_text = example['annotated_text']

    def convert_annotated_text(text):
        return re.sub(r'<(\w+)([^>]*)>([^<]*)</\1>', r'\3</\1\2>', text)

    def introduce_errors(text):
        # Randomly change some discourse types or effectiveness ratings
        def random_change(match):
            if random.random() < 0.3:  # 30% chance to introduce an error
                types = ["Lead", "Position", "Claim", "Counterclaim", "Rebuttal", "Evidence", "Concluding Statement"] # TODO: Add "Unannotated"
                effectiveness = ["Effective", "Adequate", "Ineffective"]
                new_type = random.choice(types)
                new_effectiveness = random.choice(effectiveness)
                return f"{match.group(1)}</{new_type}:{new_effectiveness}>"
            return match.group(0)
        return re.sub(r'(.*?)</.*?:.*?>', random_change, text)

    if include_annotations:
        if random.random() < 0.3:  # 30% chance for partly_marked
            essay_type = "partly_marked"
            # Keep 30-70% of annotations, introduce some errors
            keep_prob = random.uniform(0.3, 0.7)
            partly_annotated = re.sub(r'<(\w+)([^>]*)>([^<]*)</\1>',
                                      lambda m: m.group(0) if random.random() < keep_prob else m.group(3),
                                      annotated_text)
            #essay_text = convert_annotated_text(introduce_errors(partly_annotated))
            essay_text = introduce_errors(partly_annotated)

            instruction = f"""Complete the tagging for all discourse segments and correct any errors in existing tags. Use this format: "<Discourse Type> Text of discourse segment. </Discourse Type:Discourse Effectiveness>"
            Essay: {essay_text}
            """
        else:
            essay_type = "fully_marked"
            #essay_text = convert_annotated_text(introduce_errors(annotated_text))
            essay_text = introduce_errors(annotated_text)
            instruction = f"""
            Review all tagged discourse segments. Identify and correct any errors in discourse type or effectiveness rating.
            Essay: {essay_text}
            """
    else:
        essay_type = "unmarked"
        essay_text = full_text
        instruction = f"""
        Tag the start and end of discourse segments with an xml-like tag that indicates its Discourse Type and Discourse Effectiveness.
        Use this format: "<Discourse Type> Text of discourse segment. </Discourse Type:Discourse Effectiveness>"

        Example essay text: "Online learning is overwhelmingly popular. This study shows that 80% of students prefer online learning."
Example annotated text: "Online learning is overwhelmingly popular. <Evidence>This study shows that 80% of students prefer online learning.</Evidence:Adequate>"
        Essay: {essay_text}
        """

    prompt = f"""For the essay below, identify these Discourse Types:
- Lead: Attention-grabbing introduction pointing to thesis
- Position: Main opinion/conclusion
- Claim: Supports position
- Counterclaim: Opposes position/claim
- Rebuttal: Refutes counterclaim
- Evidence: Supports claims/counterclaims/rebuttals
- Concluding Statement: Restates position

Use the effctiveness ratings: Effective, Adequate, or Ineffective.
{instruction}
"""
    prompt = prompt.strip()
    return prompt

In [8]:
def create_test_prompt(example):
    full_text = example['full_text']
    prompt = f"""For the essay below, tag the start and end of discourse segments with an xml-like tag that indicates its Discourse Type and Discourse Effectiveness. Use this format. "<Discourse Type> Text of the discourse </Discourse Type:Discourse Effectiveness>"
Identify these Discourse Types:
- Lead: Attention-grabbing introduction pointing to the thesis
- Position: Main opinion/conclusion
- Claim: Supports position
- Counterclaim: Opposes position/claim
- Rebuttal: Refutes counterclaim
- Evidence: Supports claims/counterclaims/rebuttals
- Concluding Statement: Restates position

Assign a discourse effectiveness of Effective, Adequate, or Ineffective.

Example essay text: Online learning is overwhelmingly popular. This study shows that 80% of students prefer online learning.
Example annotated text: Online learning is overwhelmingly popular. <Evidence:Adequate>This study shows that 80% of students prefer online learning.</Evidence:Adequate>

Essay:
{full_text}"
"""
    prompt = prompt.strip()
    return prompt

In [9]:
train_data['prompt'] = train_data.apply(lambda row: create_train_prompt(row, include_annotations=True), axis=1)
print(train_data['prompt'][0])

eval_data['prompt'] = eval_data.apply(lambda row: create_test_prompt(row), axis=1)
print(eval_data.reset_index(drop=True)['prompt'][0])

For the essay below, identify these Discourse Types:
- Lead: Attention-grabbing introduction pointing to thesis
- Position: Main opinion/conclusion
- Claim: Supports position
- Counterclaim: Opposes position/claim
- Rebuttal: Refutes counterclaim
- Evidence: Supports claims/counterclaims/rebuttals
- Concluding Statement: Restates position

Use the effctiveness ratings: Effective, Adequate, or Ineffective.
Complete the tagging for all discourse segments and correct any errors in existing tags. Use this format: "<Discourse Type> Text of discourse segment. </Discourse Type:Discourse Effectiveness>"
            Essay: Phones

<Lead>Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. </Lead:Adequate><Position>They are some really bad consequences when stuff happens when it comes to a phone. </Position:Effective>Some certain

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
* [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

In [10]:
  def prepare_sharegpt_dataset(df):
    def create_sharegpt_conversation(index, row):
        conversation = [
            {"from": "human", "value": row['prompt']},
            {"from": "assistant", "value": row['annotated_text']}
        ]
        return {
            "id": index,
            "essay_id": row['essay_id'],
            "conversations": conversation
        }

    data = [create_sharegpt_conversation(i, row) for i, row in df.iterrows()]

    dataset = Dataset.from_list(data)
    return dataset

In [11]:
# Convert dataframes to datasets
train_dataset = prepare_sharegpt_dataset(train_data)
print(train_dataset)

eval_dataset = prepare_sharegpt_dataset(eval_data)
print(eval_dataset)

Dataset({
    features: ['id', 'essay_id', 'conversations'],
    num_rows: 3245
})
Dataset({
    features: ['id', 'essay_id', 'conversations'],
    num_rows: 812
})


In [12]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = load_model,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = userdata.get('huggingface')
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [13]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 42,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [14]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) + EOS_TOKEN for convo in convos]
    return {"text": texts}
pass

#from datasets import load_dataset
train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)
eval_dataset = eval_dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/3245 [00:00<?, ? examples/s]

Map:   0%|          | 0/812 [00:00<?, ? examples/s]

<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [15]:
batch_size = 4
gradient_steps = 4
eval_batch_size = 2
eval_steps = 20
save_directory = "/content/drive/MyDrive/LLM Projects/Kaggle/EssayGrader/Llama_Annotator_v2"

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = batch_size,
        gradient_accumulation_steps = gradient_steps,
        warmup_ratio = 0.05,
        num_train_epochs = 3, # Set this for 1 full training run.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_dir=save_directory,
        logging_steps=1,
        report_to="tensorboard",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = save_directory,

        # Evaluation settings
        fp16_full_eval = True,
        per_device_eval_batch_size = eval_batch_size,
        eval_strategy = "steps",
        eval_steps = eval_steps,

        save_strategy="steps",
        save_steps=20,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
    ),
)

Map (num_proc=2):   0%|          | 0/3245 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/812 [00:00<?, ? examples/s]

In [16]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
6.152 GB of memory reserved.


In [17]:
trainer_stats = trainer.train()

model.save_pretrained("save_directory")
tokenizer.save_pretrained("save_directory")

# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,245 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 609
 "-____-"     Number of trainable parameters = 83,886,080


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
20,1.171000,1.412106
40,0.997800,1.527083
60,1.068400,1.462923
80,1.086200,1.390952
100,0.954100,1.395466
120,0.912000,1.371199
140,1.020700,1.378765
160,0.990600,1.367798
180,0.936500,1.350469
200,0.956400,1.335453


KeyboardInterrupt: 

In [18]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

NameError: name 'trainer_stats' is not defined

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [ ]:
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = load_model, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    )
FastLanguageModel.for_inference(model) # Enable native 2x faster inference


tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) + EOS_TOKEN for convo in convos]
    return {"text": texts}
pass

#from datasets import load_dataset
train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

messages = [
    {"from": "human", "value": '''
"For the essay below, tag the start and end of discourse segments with an xml-like tag that indicates its Discourse Type and Discourse Effectiveness. Use this format. "<Discourse Type> Text of the discourse </Discourse Type:Discourse Effectiveness>"
Identify these Discourse Types:
- Lead: Attention-grabbing introduction pointing to the thesis
- Position: Main opinion/conclusion
- Claim: Supports position
- Counterclaim: Opposes position/claim
- Rebuttal: Refutes counterclaim
- Evidence: Supports claims/counterclaims/rebuttals
- Concluding Statement: Restates position

Assign a discourse effectiveness of Effective, Adequate, or Ineffective.

Example essay text: "Online learning is overwhelmingly popular. This study shows that 80% of students prefer online learning."
Example annotated text: "<Claim:Ineffective>Online learning is overwhelmingly popular.</Claim:Ineffective> <Evidence:Adequate>This study shows that 80% of students prefer online learning.</Evidence:Adequate>"

Essay:
I believe that this computer would work wonders in a classroom. It could allow teachers to know which of the children in the classroom are having problems understanding the lesson, allowing the teacher to help them more. This could also help other studys of the human anatomy, as it shows how facial expressions are done and also what they mean. In the famous, "Mona Lisa" painting, many people observe her smile and think that she is being forced to smile, and that she is actually scared, frighted, or just angry. Using this computer means that we could debunk/prove this theory. This could also solve crimes, some that have not been solved to this day!

This computer could also help with crimes as it could show that if a suspect is making a certain facial expression, they are lying or telling the truth. This could help police catch people who are breaking the law, but not put others in prison for nothing. This computer could help with these problems that occur in the real world, and it would save lives in the process. This would be very helpful in peoples daily lives, and it could help people who have problem such as mental health. If a child is slower than other children, this test could help decide whether or whether not these children need the extra help. If they do need help, this machine could even put them in the correct classes!"
'''},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = max_seq_length, use_cache = True)
tokenizer.batch_decode(outputs)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>